# Weekly project

Today you are going to implement the last parts of the algorithm you started on monday. For reference you can see it below.

![title](algorithm_3.png)

It is a good idea to follow and track the steps in the algorithm in the below implementation. Only take one step at a time.

Once you have the algorithm up and running you can try with a larger dataset to see if your algorithm is able to maintain good accurracy over a longer distance. The larger dataset can be found here:
[Left images](https://dtudk-my.sharepoint.com/:u:/g/personal/evanb_dtu_dk/EQu8kmGBDDROtGJ7IkZB2tQBJrxmgY9t8LVM_JuEi83TYw)
[Right images](https://dtudk-my.sharepoint.com/:u:/g/personal/evanb_dtu_dk/EcKI_zrXTvpMulizidCZm4oBLJcQ_LTV9Zs6oQFF74JTRQ)

In [ ]:
import numpy as np
import cv2 as cv2
from numpy.linalg import inv, pinv
import matplotlib.pyplot as plt
import time as t
from helpers import *

def extract_keypoints_surf(img1, img2, K, baseline, refPoints = None):
    """
    use surf to detect keypoint features
    remember to include a Lowes ratio test
    """
    # img1, img2 have to be GRAYSCALE image to use SURF
    #-- Step 1: Detect the keypoints using SURF Detector, compute the descriptors
    detector = cv2.xfeatures2d.SURF_create()
    key_points1, descriptors1 = detector.detectAndCompute(img1,None)
    key_points2, descriptors2 = detector.detectAndCompute(img2,None)
    
    #-- Step 2: Matching descriptor vectors using brute force
    #     bf = cv2.BFMatcher() #obsolete
#     bf = cv2.BFMatcher_create()
#     matches = bf.knnMatch(descriptors1,descriptors2, k=2)
#     match_points1, match_points2 = [],[]
#     # Apply ratio test
#     for m,n in matches:
#         if m.distance < 0.75*n.distance:
#             match_points1.append([m])
#             match_points2.append([n])
    
    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict()   # or pass empty dictionary
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(descriptors1,descriptors2,k=2)

    # ratio test as per Lowe's paper
    match_points1, match_points2 = [], []
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.7*n.distance:
            match_points1.append(key_points1[m.queryIdx].pt)
            match_points2.append(key_points2[m.trainIdx].pt)
            
    p1 = np.array(match_points1).astype(float)
    p2 = np.array(match_points2).astype(float)
    
    if refPoints is not None:
        mask = removeDuplicate(p1, refPoints)
        p1 = p1[mask,:]
        p2 = p2[mask,:]

    ##### ############# ##########
    ##### Do Triangulation #######
    ##### ########################
    #project the feature points to 3D with triangulation
    
    #projection matrix for Left and Right Image
    M_left = K.dot(np.hstack((np.eye(3), np.zeros((3, 1)))))
    M_rght = K.dot(np.hstack((np.eye(3), np.array([[-baseline, 0, 0]]).T)))

    p1_flip = np.vstack((p1.T, np.ones((1, p1.shape[0]))))
    p2_flip = np.vstack((p2.T, np.ones((1, p2.shape[0]))))
    
    # This function reconstructs 3-dimensional points y using their observations with a stereo camera.
    # arg: proj_matrixL, proj_matrixR, 2xN array of features points in the first image and second
    # return: 4xN array of reconstructed points in homogenous cordinates in the world's cordinate frame
    P = cv2.triangulatePoints(M_left, M_rght, p1_flip[:2], p2_flip[:2])

    # Normalize homogeneous coordinates (P->Nx4  [N,4] is the normalizer/scale)
    P = P / P[3]
    land_points = P[:3]

    return land_points.T, p1

def featureTracking(prev_img, next_img, prev_points, world_points):
    """
    fill in
    """
    params = dict(winSize=(21, 21),
                 maxLevel=3,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
    
    #next_points, status, _ = ...
    # prev_img and next_img should be grayscale (8bit)
    next_points, status, _ = cv2.calcOpticalFlowPyrLK(prev_img, next_img, prev_points, None, **params)
    status = status.reshape(status.shape[0])
    prev_points = prev_points[status==1]
    next_points = next_points[status==1]
    world_points = world_points[status==1]

    return world_points, prev_points, next_points

def playImageSequence(left_img, right_img, K):

    baseline = 0.54;

    ##### ################################# #######
    ##### Get 3D points Using Triangulation #######
    ##### #########################################
    """
    Implement step 1.2 and 1.3
    Store the features in 'reference_2D' and the 3D points (landmarks) in 'landmark_3D'
    hint: use 'extract_keypoints_surf' above
    """
    points, p1 = extract_keypoints_surf(left_img, right_img, K, baseline)
    p1 = p1.astype('float32')

    # reference
    reference_img = left_img
    reference_2D = p1
    landmark_3D = points

    # Groundtruth for plot
    truePose = getTruePose()
    traj = np.zeros((600, 600, 3), dtype=np.uint8);
    maxError = 0

    for i in range(0, 101): #number of images
        print('image: ', i)
        curImage = getLeftImage(i)
        curImage_R = getRightImage(i)

        ##### ############################################################# #######
        ##### Calculate 2D and 3D feature correspndances in t=T-1, and t=T  #######
        ##### #####################################################################
        """
        Implement step 2.2)
        Remember this is a part of a loop, so the initial features are already
        provided in step 1)-1.3) outside the loop in 'reference_2D' and 'landmark_3D'
        """
        landmark_3D, reference_2D, tracked_2Dpoints = featureTracking(reference_img, curImage, reference_2D, landmark_3D)
        
        ##### ################################# #######
        ##### Calculate relative pose using PNP #######
        ##### #########################################
        """
        Implement step 2.3)
        """
        _, rvec, tvec, inliers = cv2.solvePnPRansac(landmark_3D, tracked_2Dpoints, K, None)
        
        ##### ####################################################### #######
        ##### Get Pose and Tranformation Matrix in world coordionates #######
        ##### ###############################################################
        rot, _ = cv2.Rodrigues(rvec)
        tvec = -rot.T.dot(tvec)  # coordinate transformation, from camera to world. What is the XYZ of the camera wrt World
        inv_transform = np.hstack((rot.T, tvec))  # inverse transform. A tranform projecting points from the camera frame to the world frame

        ##### ################################# #######
        ##### Get 3D points Using Triangulation #######
        ##### #########################################
        # re-obtain the 3D points
        """
        Implement step 2.4)
        """
        curImage_R = getRightImage(i)
        landmark_3D_new, reference_2D_new  = extract_keypoints_surf(curImage, curImage_R, K, baseline, reference_2D)
        
        #Project the points from camera to world coordinates
        reference_2D = reference_2D_new.astype('float32')
        landmark_3D = inv_transform.dot(np.vstack((landmark_3D_new.T, np.ones((1, landmark_3D_new.shape[0])))))
        landmark_3D = landmark_3D.T

        ##### ####################### #######
        ##### Done, Next image please #######
        ##### ###############################
        reference_img = curImage

        ##### ################################## #######
        ##### START OF Print and visualize stuff #######
        ##### ##########################################
        # draw images
        draw_x, draw_y = int(tvec[0]) + 300, 600-(int(tvec[2]) + 100);
        true_x, true_y = int(truePose[i][3]) + 300, 600-(int(truePose[i][11]) + 100)

        curError = np.sqrt(
            (tvec[0] - truePose[i][3]) ** 2 +
            (tvec[1] - truePose[i][7]) ** 2 +
            (tvec[2] - truePose[i][11]) ** 2)
        
        if (curError > maxError):
            maxError = curError

        print(tvec[0],tvec[1],tvec[2], rvec[0], rvec[1], rvec[2])
        print([truePose[i][3], truePose[i][7], truePose[i][11]])
        
        text = "Coordinates: x ={0:02f}m y = {1:02f}m z = {2:02f}m".format(float(tvec[0]), float(tvec[1]),
                                                                           float(tvec[2]));
        cv2.circle(traj, (draw_x, draw_y), 1, (0, 0, 255), 2);
        cv2.circle(traj, (true_x, true_y), 1, (255, 0, 0), 2);
        cv2.rectangle(traj, (10, 30), (550, 50), (0, 0, 0), cv2.FILLED);
        cv2.putText(traj, text, (10, 50), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1, 8);

        h1, w1 = traj.shape[:2]
        h2, w2 = curImage.shape[:2]
        vis = np.zeros((max(h1, h2), w1 + w2, 3), np.uint8)
        vis[:h1, :w1, :3] = traj
        vis[:h2, w1:w1 + w2, :3] = np.dstack((np.dstack((curImage,curImage)),curImage))

        cv2.imshow("Trajectory", vis);
        k = cv2.waitKey(1) & 0xFF
        if k == 27: break


    cv2.waitKey(0)
    cv2.destroyAllWindows()
    print('Maximum Error: ', maxError)
    ##### ################################ #######
    ##### END OF Print and visualize stuff #######
    ##### ########################################

if __name__ == '__main__':
    left_img = getLeftImage(0)
    right_img = getRightImage(0)

    K = getK()

    playImageSequence(left_img, right_img, K)

image:  0
[-0.00096162] [-0.00106012] [-0.00067494] [-4.76261941e-05] [-7.27946042e-05] [-4.61530021e-05]
[5.551115e-17, 3.330669e-16, -4.440892e-16]
image:  1
[-0.01883713] [-0.02438154] [0.69928025] [-0.00131243] [0.00297027] [-0.00349888]
[-0.04690294, -0.02839928, 0.8586941]
image:  2
[-0.02520413] [-0.03002088] [1.39640133] [-0.00290558] [0.00720221] [-0.00217907]
[-0.09374345, -0.05676064, 1.716275]
image:  3
[-0.04437948] [-0.03466033] [2.1145731] [-0.00457627] [0.01077964] [-0.00209991]
[-0.1406429, -0.08515762, 2.574964]
image:  4
[-0.0669303] [-0.04299374] [2.84704604] [-0.00528294] [0.01546029] [-0.0006]
[-0.1874858, -0.1135202, 3.432648]
image:  5
[-0.08663507] [-0.05564742] [3.59479608] [-0.00550457] [0.01968951] [-0.00161931]
[-0.2343818, -0.141915, 4.291335]
image:  6
[-0.11106644] [-0.06420737] [4.35521724] [-0.00706052] [0.02422043] [0.00040925]
[-0.2812195, -0.1702743, 5.148987]
image:  7
[-0.14085747] [-0.07187916] [5.13059143] [-0.00800461] [0.02934418] [0.0032038]


[-3.77426971] [-0.8728371] [59.52029833] [-0.00781498] [0.06865291] [0.00340481]
[-3.641784, -2.021539, 61.09125]
image:  66
[-3.85082005] [-0.89274786] [60.4284424] [-0.01300296] [0.0691423] [0.00576195]
[-3.703384, -2.049159, 62.00632]
image:  67
[-3.93083523] [-0.91469624] [61.34666568] [-0.01265792] [0.06989385] [0.01128521]
[-3.776035, -2.078109, 62.91678]
image:  68
[-4.00578748] [-0.94188617] [62.23698617] [-0.01058363] [0.0706986] [0.01416513]
[-3.84394, -2.112094, 63.81519]
image:  69
[-4.07939928] [-0.98495844] [63.12242695] [-0.00862702] [0.07152685] [0.01739383]
[-3.912602, -2.145353, 64.69786]
image:  70
[-4.15648793] [-1.01013649] [63.98958126] [-0.00905681] [0.0723739] [0.01883144]
[-3.978228, -2.173835, 65.56672]
image:  71
[-4.23448162] [-1.03736162] [64.83990623] [-0.00839956] [0.07361287] [0.02139803]
[-4.046625, -2.201409, 66.42163]
image:  72
[-4.30656049] [-1.06158612] [65.68579589] [-0.00788338] [0.07492211] [0.02237446]
[-4.114442, -2.230822, 67.26455]
image:  7

# Challenge 
The current implementation only uses features computed at the current timestep. However, as we process more images we potentially have a lot of features from previous timesteps that are still valid. The challenge is to expand the `extract_keypoints_surf(..., refPoints)` function by giving it old reference points. You should then combine your freshly computed features with the old features and remove all duplicates. This requires you to keep track of old features and 3D points.

Hint 1: look in `helpers.py` for removing duplicates.

Hint 2: you are not interested in points that are behind you, so remember to remove points that are negative in the direction you move.